# Counting Experiments, Interrupts

In this project we'll explore the practical aspects of several types of "counting" experiments.

One of the most common scenarios involves some kind of event that occurs periodically
that you'd like to capture. A simple example of this is the detection of a nuclear decay process.
There are naturally occuring atomic nuclei in nature that are not "stable". Some of these
nuclei end up in [products](https://www.thoughtco.com/how-radioactive-is-fiesta-ware-608648) that 
you may have [encountered](http://large.stanford.edu/courses/2011/ph241/eason1/) in your daily life.
Others can be applied in medical treatments or diagnostic equipment. You could write a program
that measures the time between pulses from a Gieger-Muller Tube (GMT) exposed to a radioactive source.

You could, for example, vary the distance to the source and see the effect on the count rate.

The output of the GMT is normally 5V. When the GMT detects a particle it will emit a short pulse on its output pin during which the pin voltage drops momentarily to zero, and then quickly, back to 5V. We can set up an "interrupt" routine on the Arduino that runs only when this event occurs. You can "attach" an interrupt routine to a particular hardware event using "[attachInterrupt](https://www.arduino.cc/reference/en/language/functions/external-interrupts/attachinterrupt/)" like so:

    attachInterrupt(digitalPinToInterrupt(pin number), ISR function name, interrupt mode);
    
Please [see the link](https://www.arduino.cc/reference/en/language/functions/external-interrupts/attachinterrupt/) for details, and an example. Note in particular that different Arduino boards allow interrupts on only specific pins (e.g., the RedBoard only allows pins 2 and 3 for external interrupts).
    
Suppose we want to count every time a falling edge pulse comes in on pin 3. We could set this up with:

    attachInterrupt(digitalPinToInterrupt(3), myFunc, FALLING);
    
so long as somewhere (earlier) in the program we declared:

    volatile unsigned long count;

    void myFunc() {
        count++;
    }

then, in the loop() function we could say:

    void loop() {
        long int copy; // a non-volatile local copy of the current count
        
        delay(1000);
        Serial.print(millis());
        Serial.print(",");
        
        noInterrupts();
        copy = count; // make a copy with interrupts disabled
        interrupts();
        
        Serial.println(copy);
    }
    
and the program would print the 'current' count every second or so.

Here's a full example illustrating counting with visual feedback through the on-board LED:

    volatile long int count = 0L;

    void setup()
    {
      pinMode(13, OUTPUT);
      Serial.begin(9600);
      attachInterrupt(digitalPinToInterrupt(2), click, FALLING);
    }

    void click() {
      count++;
      digitalWrite(13, ((count>>3) & 0x01));
    }

    void loop()
    {
      long int copy;
      delay(1000); // Wait for 1000 millisecond(s)
      
      noInterrupts(); // disable interrupts to make a copy
      copy = count;   // make a copy!
      count = 0L;     // zero the volatile variable.
      interrupts();   // re-enable interrupts
      
      Serial.println(copy);
    }

Deliverable: Use `attachInterrupt` to count something interesting. Surprise me!
